### Hanoi mit Klassen II
Fortseztung von `Hanoi_mit_Klassen_I.ipynb`. Nachstehend importieren wir
die dort geschriebenen Klassen `Game`  und `View` aus den Files `game.py` und `views.py`.

In [ ]:
from game import Game
from views import CanvasView

game = Game()
view = CanvasView(game)
game.new_game()
view

In [ ]:
game.move_disk(0, 2)

### Die Klasse Controller
Wir wollen das Spiel mit der Tastatur steuern k&ouml;nnen ('n': neues Spiel, '13': Scheibe von Stapel 1 auf 3 verschieben).
Wir benutzen das Canvas-Objekt der View um auf Tastendr&uuml;cke zu h&ouml;ren.

Der Controller hat ein Klassenattribut `err_out`, welches ein
Output-Widget enth&auml;lt. Um Fehlermeldungen der Methode `on_key_down` dorhin umleiten zu k&ouml;nnen,
muss `err_out` zum Zeitpunkt der Funktionsdefinition bereits definiert sein.  
Der `__init__` Methode des Controllers muss nat&uuml;rlich das Canvas-Objekt der View &uuml;bergeben werden, 
damit auf Tastendr&uuml;cke geh&ouml;rt werden kann, und ebenso eine Game-Instanz, damit die
via Tastatur erhaltenen Befehle an die Game-Instanz weitergegeben werden k&ouml;nnen.

**Beachte**: Verschiedene Controller-Instanzen benutzen das gleiche Output-Widget.
Das ist kein Problem, wir brauchen das Output-Widget ja nur zum Debuggen.


Man k&ouml;nnte den Controller auch direkt in die View integrieren (siehe Aufgabe unten).

In [ ]:
import widgets_helpers
import canvas_helpers
from IPython.display import display


class Controller:

    err_out = widgets_helpers.new_output()

    def __init__(self, game, canvas):
        self.game = game
        self.canvas = canvas
        self.src = None

        canvas_helpers.remove_all_callbacks(self.canvas)
        self.canvas.on_key_down(self.on_key_down)

    @err_out.capture()
    def on_key_down(self, key, *flags):
        # print(key)
        if key in '123' and self.src is None:
            self.src = int(key) - 1
            return
        elif key in '123':
            dst = int(key) - 1
            self.game.move_disk(self.src, dst)
        elif key == 'n':
            self.game.new_game()
        self.src = None

    def _ipython_display_(self):
        display(self.canvas, self.err_out)

In [ ]:
# Scheiben lassen sich mit Tastatur ('12', ...) verschieben
controller = Controller(game, view.canvas)
controller

In [ ]:
# Error triggern
controller.on_key_down()

In [ ]:
# err_out des Controllers loeschen
Controller.err_out.clear_output()

### Direkte Verwendung der Funktion `Controller.on_key_down` als Callback
Die Funktion  
`Controller.on_key_down(self, key, *flags)`  
ist eine normale Funktion mit 3 Argumenten.
Wir wollen daraus eine Funktion

```python
def on_key_down(key, *flags):
    ...
```

fabrizieren, die wir als Callback f&uuml;r das `on_key_down` Event registrieren k&ouml;nnen.
Die Funktion  
`Controller.on_key_down(self, key, *flags)`  
benutzt das `self` Argument um mit `self.game` auf die Game-Instanz zuzugreifen und um
den Ausgangsstapel in `self.src` zu speichern.
Unsere Funktion `on_key_down(key, *flags)` benutzt statt `self` eine Klasse `C`,
welche den Zugriff auf die Game-Instanz und eine Variable `src` erlaubt.

In [ ]:
class C:
    src = None
    game = game


def on_key_down(key, *flags):
    Controller.on_key_down(C, key, *flags)

In [ ]:
# canvas Attribut der View und err_out des Controllers anzeigen
display(view.canvas, Controller.err_out)

In [ ]:
# oben definierte Funktion als Callback registrieren
canvas_helpers.remove_all_callbacks(view.canvas)
view.canvas.on_key_down(on_key_down)

### Aufgabe: Integriere den Controller in die View
Copy/paste die Methode `on_mouse_down` des Controllers in die View.
Damit die Dekoration funktioniert, muss die View &uuml;ber ein
Klassenattribut `err_out` verf&uuml;gen.
Weiter brauchen View-Instanzen ein Attribut `src`.

Registriere nun das Callback `on_mouse_down`  in  `__init__`.

### Aufgabe: Erweitere den Controller
Erweitere die Klasse `Controller` um Methoden
`on_mouse_down` und `on_mouse_up`, welche das Verschieben der Scheiben mit der Maus 
m&ouml;glich machen sollen. Orientiere dich an der Maussteuerung im Notebook `Tuerme_von_Hanoi.ipynb` der Lektion 12.
Passe dann die `__init__`-Methode entsprechend an, um die neuen Eventhandlers zu registrieren.



```python

    def __init__(self, game, canvas):
        ...
        self.canvas.on_key_down(self.on_key_down)
        self.canvas.on_mouse_down(self.on_mouse_down)
        self.canvas.on_mouse_up(self.on_mouse_up)
        ...

    @err_out.capture()
    def on_mouse_down(self, x, y):
        ...

    @err_out.capture()
    def on_mouse_up(self, x, y):
        ...
```

In [ ]:
from Loesungen_13.KeyMouseController import KeyMouseController


controller = KeyMouseController(game, view.canvas)
game.new_game()
controller